In [260]:
import re

In [261]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
color = sns.color_palette()

%matplotlib inline


In [355]:
tdf=pd.read_csv('./train 3.csv');train_df=tdf[:];test_df=pd.read_csv('./test 3.csv')

In [415]:
tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,4))
full_tfidf = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

In [232]:
tfidf_vec1 = TfidfVectorizer(stop_words='english', ngram_range=(1,4))
full_tfidf11 = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf1 = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf1 = tfidf_vec.transform(test_df['text'].values.tolist())

In [233]:
def splitter(x,g):
    return(re.compile(g).split(x))

In [234]:
def feature_extractor(x):
    #return(ngram_feature_extractor(x,3))
    a1=x.split()
    a2=splitter(x,',.;')
    a3=splitter(x,'"')
    a4=splitter(x,"ABCDEFGHIJKLMNOPQRSTUVWXYZ");
    a5=float(len(x))/float(len(a1));
    a6=splitter(x,"0123456789")
    a7=splitter(x,'><*&^%$#@~`')
    a8=splitter(x,"!?")
    a9=x.split("i ")
    a10=x.split(", and")
    a11=x.split("greek")
    a12=x.split("what ")
    a13=x.split("however ")
    a14=len(x)
    return([a14,len(a1),len(a2)/float(len(a1)),a5,len(a9)/float(len(a1)),len(a10)/float(len(a1)),len(a11)/float(len(a1)),len(a12)/float(len(a1)),len(a13)/float(len(a1))])
    #return([len(a1),len(a2)/float(len(a1)),a5,len(a6)/float(len(a1)),len(a7)/float(len(a1)),len(a8)/float(len(a1)),len(a9)/float(len(a1)),len(a10)/float(len(a1)),len(a11)/float(len(a1))])
    #return([len(a1),len(a2),len(a3),len(a4),a5,len(a6),len(a7),len(a8),len(a9),len(a10),len(a11)])

In [235]:
train_extra_features=scipy.sparse.csr_matrix([feature_extractor(m) for m in train_df['text']]);
test_extra_features=scipy.sparse.csr_matrix([feature_extractor(m) for m in test_df['text']]);

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: split() requires a non-empty pattern match.
  


In [236]:
import scipy

In [288]:
test_extra_features.shape

(8392, 9)

In [238]:
train_tfidf1[0,1]

0

In [239]:
#train_tfidf=scipy.sparse.hstack((train_tfidf,train_tfidf1));
train_tfidf=scipy.sparse.hstack((train_tfidf,train_extra_features));

In [240]:
#test_tfidf=scipy.sparse.hstack((test_tfidf,test_tfidf1));
test_tfidf=scipy.sparse.hstack((test_tfidf,test_extra_features));

In [416]:
train_tfidf=scipy.sparse.csr_matrix(train_tfidf);test_tfidf=scipy.sparse.csr_matrix(test_tfidf)

In [401]:
train_tfidf=scipy.sparse.hstack((train_tfidf,wq0*1))

In [399]:
test_tfidf=scipy.sparse.hstack((test_tfidf,wq0*1))

In [417]:
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

In [418]:
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = train_df['author'].map(author_mapping_dict)

In [419]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])
kf = model_selection.KFold(n_splits=500, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(tdf):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 500.

Mean cv score :  0.435300821639


In [184]:
pred_full_test.shape

(8392, 3)

In [277]:
0.9345-0.9345%0.01

0.93

In [282]:
pred_full_test2=pred_full_test*0.8+0.2*pred_full_test1

In [283]:
pred_full_test2=pred_full_test2-pred_full_test2%0.001

In [284]:
sub1=pd.read_csv("./sample_submission 2.csv")

In [285]:
sub1['EAP']=pred_full_test2[:,0];sub1['HPL']=pred_full_test2[:,1];sub1['MWS']=pred_full_test2[:,2]

In [287]:
sub1.to_csv("./sub55.csv",index=False)

In [186]:
np.mean([len(s) for s in train_df['text']])

149.05740844782676

In [189]:
np.mean([len(s) for s in train_df['text'][train_df['author']=='EAP']])

142.22594936708862

In [190]:
np.mean([len(s) for s in train_df['text'][train_df['author']=='MWS']])

151.65982792852415

In [191]:
np.mean([len(s) for s in train_df['text'][train_df['author']=='HPL']])

155.84347826086957

In [248]:
def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.3):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 10
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = child
    param['subsample'] = 0.8
    param['colsample_bytree'] = colsample
    param['seed'] = seed_val
    param['nthread'] = 1000
    #param['eval_metric'] = 'mlogloss'
    num_rounds = 2000

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest, ntree_limit = model.best_ntree_limit)
    if test_X2 is not None:
        xgtest2 = xgb.DMatrix(test_X2)
        pred_test_y2 = model.predict(xgtest2, ntree_limit = model.best_ntree_limit)
    return pred_test_y, pred_test_y2, model

In [249]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])
for dev_index, val_index in kf.split(train_tfidf):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, test_tfidf, seed_val=0)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("cv scores : ", cv_scores)

[0]	train-mlogloss:1.08497	test-mlogloss:1.08754
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
[20]	train-mlogloss:0.935596	test-mlogloss:0.965599
[40]	train-mlogloss:0.861496	test-mlogloss:0.906483
[60]	train-mlogloss:0.810595	test-mlogloss:0.868511
[80]	train-mlogloss:0.771628	test-mlogloss:0.839387
[100]	train-mlogloss:0.73746	test-mlogloss:0.816761
[120]	train-mlogloss:0.707674	test-mlogloss:0.797166
[140]	train-mlogloss:0.682576	test-mlogloss:0.780502
[160]	train-mlogloss:0.659913	test-mlogloss:0.765866
[180]	train-mlogloss:0.639598	test-mlogloss:0.752304
[200]	train-mlogloss:0.619699	test-mlogloss:0.741121
[220]	train-mlogloss:0.602393	test-mlogloss:0.730281
[240]	train-mlogloss:0.586206	test-mlogloss:0.721803
[260]	train-mlogloss:0.571622	test-mlogloss:0.711835
[280]	train-mlogloss:0.557777	test-mlogloss:0.70344
[300]	train-mlogloss:0.544767	test-mlogloss:0.69628
[320]	train-

[1260]	train-mlogloss:0.270672	test-mlogloss:0.57509
[1280]	train-mlogloss:0.267888	test-mlogloss:0.574587
[1300]	train-mlogloss:0.264965	test-mlogloss:0.573906
[1320]	train-mlogloss:0.261982	test-mlogloss:0.573737
[1340]	train-mlogloss:0.25934	test-mlogloss:0.573126
[1360]	train-mlogloss:0.256585	test-mlogloss:0.572792
[1380]	train-mlogloss:0.25386	test-mlogloss:0.572268
[1400]	train-mlogloss:0.251179	test-mlogloss:0.571675
[1420]	train-mlogloss:0.248186	test-mlogloss:0.571792
[1440]	train-mlogloss:0.245716	test-mlogloss:0.571055
[1460]	train-mlogloss:0.243095	test-mlogloss:0.571102
[1480]	train-mlogloss:0.240524	test-mlogloss:0.571236
[1500]	train-mlogloss:0.238364	test-mlogloss:0.570765
[1520]	train-mlogloss:0.236083	test-mlogloss:0.570553
[1540]	train-mlogloss:0.233493	test-mlogloss:0.570206
[1560]	train-mlogloss:0.231224	test-mlogloss:0.570135
[1580]	train-mlogloss:0.228899	test-mlogloss:0.570236
[1600]	train-mlogloss:0.226619	test-mlogloss:0.570089
Stopping. Best iteration:
[1556

[720]	train-mlogloss:0.383545	test-mlogloss:0.596781
[740]	train-mlogloss:0.377923	test-mlogloss:0.594688
[760]	train-mlogloss:0.372687	test-mlogloss:0.593097
[780]	train-mlogloss:0.367861	test-mlogloss:0.591264
[800]	train-mlogloss:0.362825	test-mlogloss:0.589474
[820]	train-mlogloss:0.357861	test-mlogloss:0.587698
[840]	train-mlogloss:0.352689	test-mlogloss:0.585781
[860]	train-mlogloss:0.348391	test-mlogloss:0.583978
[880]	train-mlogloss:0.343833	test-mlogloss:0.582336
[900]	train-mlogloss:0.339674	test-mlogloss:0.580706
[920]	train-mlogloss:0.335067	test-mlogloss:0.579154
[940]	train-mlogloss:0.33088	test-mlogloss:0.577387
[960]	train-mlogloss:0.327086	test-mlogloss:0.576302
[980]	train-mlogloss:0.323222	test-mlogloss:0.574981
[1000]	train-mlogloss:0.318961	test-mlogloss:0.574058
[1020]	train-mlogloss:0.315243	test-mlogloss:0.572867
[1040]	train-mlogloss:0.311479	test-mlogloss:0.57187
[1060]	train-mlogloss:0.308088	test-mlogloss:0.57057
[1080]	train-mlogloss:0.304733	test-mlogloss:

In [ ]:
pred_full_test.shape

In [252]:
pred_full_test1=pred_full_test/5.0

In [251]:
sum(pred_full_test[0])

4.9999999403953552

In [308]:
mj=np.array([((train_y.values==i).reshape((1,19579))*train_extra_features)[0] for i in range(3)])/19579.0

In [293]:
train_extra_features.shape

(19579, 9)

In [312]:
[print(mj[:,i]) for i in range(9)]

[ 57.38725165  44.85305685  46.81709995]
[ 10.26584606   8.00097043   8.46366004]
[ 0.02559255  0.01365698  0.01695337]
[ 2.25210905  1.6053206   1.71119467]
[ 0.02574898  0.01380624  0.01698756]
[ 0.03040576  0.01771175  0.02114849]
[ 0.02559255  0.01365698  0.01695337]
[ 0.02642864  0.0143596   0.01739578]
[ 0.02562745  0.01365965  0.01700985]


[None, None, None, None, None, None, None, None, None]

In [319]:
np.vstack([(train_extra_features[:,6:7]>0.015),(train_extra_features[:,6:7]>0.020)]).shape

(2, 1)

In [317]:
(train_extra_features[:,6]>0.015).shape

(19579, 1)

In [329]:
mj=mj.transpose()

In [330]:
mj1=(mj[:,1]+mj[:,2])/2.0

In [331]:
mj0=(mj[:,1]+mj[:,0])/2.0

In [400]:
wq0=test_extra_features>mj0

In [353]:
wq1=test_extra_features>mj1

In [354]:
wq2=scipy.sparse.hstack((scipy.sparse.csr_matrix(wq0),scipy.sparse.csr_matrix(wq1)))

In [351]:
wq.shape

(19579, 18)

In [339]:
wq[5]

matrix([[ True,  True, False,  True, False,  True, False, False, False]], dtype=bool)

In [345]:
(train_extra_features[5]).toarray()

array([[  4.68000000e+02,   8.30000000e+01,   1.20481928e-02,
          5.63855422e+00,   1.20481928e-02,   2.40963855e-02,
          1.20481928e-02,   1.20481928e-02,   1.20481928e-02]])

In [346]:
mj0

array([  5.11201542e+01,   9.13340824e+00,   1.96247670e-02,
         1.92871483e+00,   1.97776057e-02,   2.40587528e-02,
         1.96247670e-02,   2.03941205e-02,   1.96435502e-02])

In [420]:
sf=pd.read_csv("/Users/souvicchakraborty/Downloads/sub_fe.csv")

In [421]:
sf.keys()

Index(['id', 'EAP', 'HPL', 'MWS'], dtype='object')

In [422]:
sf['EAP']=0.9*sf['EAP']+pred_full_test[:,0]*0.1;sf['MWS']=0.9*sf['MWS']+pred_full_test[:,2]*0.1;sf['HPL']=0.9*sf['HPL']+pred_full_test[:,1]*0.1

In [423]:
sf.to_csv("./subc0.3_0.99.csv",index=False)